In [1]:
import pdfplumber
import pandas as pd
import sys
import os
import contextlib

# pdf_path = r"C:\Users\dpere\Documents\python-projects\web-gis-scraper-desktop-electron\backend\101-0925198_תדפיס הוראות התכנית_חתום לאישור_1.pdf"
pdf_path = r"C:\Users\dpere\Documents\python-projects\web-gis-scraper-desktop-electron\backend\101-0135004_תדפיס הוראות התכנית_חתום לאישור_1.pdf"

search_phrase = "טבלת זכויות והוראות בניה"

In [2]:
@contextlib.contextmanager
def suppress_stderr():
    with open(os.devnull, "w") as devnull:
        old_stderr = sys.stderr
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stderr = old_stderr

In [3]:
def find_table_start_page(pdf_path: str, phrase: str) -> int | None:
    with suppress_stderr():
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text and phrase in text:
                    return i
        return None

In [4]:
def extract_raw_table_from_pdf(pdf_path: str, start_page: int) -> pd.DataFrame:
    """
    מחלץ טבלה גולמית מ־PDF החל מעמוד מסוים.
    ממשיך עמוד אחר עמוד עד שאין טבלאות.
    לא מניח מראש את שמות העמודות – מחזיר DataFrame גולמי.
    """
    all_rows = []

    with suppress_stderr():
        with pdfplumber.open(pdf_path) as pdf:
            page_index = start_page
            while page_index < len(pdf.pages):
                page = pdf.pages[page_index]
                tables = page.extract_tables()

                if not tables:
                    break

                return tables

In [5]:
start_page = find_table_start_page(pdf_path, search_phrase[::-1])

In [6]:
table = extract_raw_table_from_pdf(pdf_path, start_page)
row1 = table[1][0]
row2 = table[1][1]
row3 = table[1][2]
row4 = table[1][3]

In [7]:
def generate_column_names_from_three_rows(
    row1: list, row2: list, row3: list
) -> list[str]:
    """
    מחזירה שמות עמודות לפי שלוש שורות כותרת, עם היפוך טקסט מלא לעברית.
    """

    def clean(val):
        return str(val).replace("\n", " ").strip() if val else None

    def reverse(val):
        return val[::-1] if val else val

    def identify_split_columns_with_mapping(row: list) -> dict:
        split_columns = []
        main_columns = []
        split_to_main = {}
        temp_main = None
        temp_splits = []

        for idx, cell in enumerate(row):
            if cell is not None:
                if temp_main is not None and temp_splits:
                    main_columns.append(temp_main)
                    for split_idx in temp_splits:
                        split_to_main[split_idx] = temp_main
                temp_main = idx
                temp_splits = []
            else:
                split_columns.append(idx)
                temp_splits.append(idx)

        if temp_main is not None and temp_splits:
            main_columns.append(temp_main)
            for split_idx in temp_splits:
                split_to_main[split_idx] = temp_main

        return {
            "main_columns": main_columns,
            "split_columns": split_columns,
            "split_to_main": split_to_main,
        }

    mapping = identify_split_columns_with_mapping(row1)
    split_to_main = mapping["split_to_main"]

    column_names = []
    for idx in range(len(row1)):
        main_val = clean(row1[idx])
        mid_val = clean(row2[idx]) if idx < len(row2) else None
        sub_val = clean(row3[idx]) if idx < len(row3) else None

        if idx in split_to_main:
            main_idx = split_to_main[idx]
            main_val = clean(row1[main_idx])
            mid_val = clean(row2[idx]) or clean(row2[main_idx])
            sub_val = clean(row3[idx]) or clean(row3[main_idx])

        parts = [main_val, mid_val, sub_val]
        parts = [reverse(p) for p in parts if p]
        name = " / ".join(parts)
        column_names.append(name)

    return column_names


column_names = generate_column_names_from_three_rows(row1, row2, row3)

In [ ]:
import re


def is_hebrew(text: str) -> bool:
    return bool(re.search(r"[\u0590-\u05FF]", text))


def clean_cell(cell: str | None) -> str | None:
    if not cell:
        return None

    # ניקוי בסיסי
    cell = str(cell).replace("\n", " ").strip()

    # הסרה של מילות רעש (כמו "מונה", "מונה הדפסה" וכו')
    junk_words = ["מונה", "הנומ", "מונה הדפסה", "הצפסה הנומ"]
    for junk in junk_words:
        cell = cell.replace(junk, "").strip()

    # אם אחרי ההסרה לא נשאר כלום – נוותר על התא
    if not cell:
        return None

    # הפוך רק אם הטקסט בעברית
    return cell[::-1] if is_hebrew(cell) else cell


def extract_table_rows_after_headers(pdf_path: str, start_page: int) -> list[list[str]]:
    rows = []
    with suppress_stderr():
        with pdfplumber.open(pdf_path) as pdf:
            page_index = start_page
            while page_index < len(pdf.pages):
                page = pdf.pages[page_index]
                tables = page.extract_tables()

                if not tables:
                    break  # אין טבלה – מפסיקים לקרוא

                for table in tables:
                    if len(table) <= 3:
                        continue  # פחות מ־3 שורות – אין ערכים

                    # אם זו הפעם הראשונה – נדלג על שלוש כותרות
                    raw_rows = table[3:] if page_index == start_page else table

                    for row in raw_rows:
                        cleaned_row = [clean_cell(cell) for cell in row]
                        rows.append(cleaned_row)

                page_index += 1
    return rows

In [9]:
search_phrase = "טבלת זכויות והוראות בניה"[::-1]
start_page = find_table_start_page(pdf_path, search_phrase)

if start_page is not None:
    data_rows = extract_table_rows_after_headers(pdf_path, start_page)

In [9]:
df = pd.DataFrame(data_rows, columns=column_names)

NameError: name 'data_rows' is not defined

In [15]:
import pdfplumber
import contextlib
import os
import sys
import pandas as pd


class PdfTableExtractor:
    @staticmethod
    def extract_table_from_pdf(pdf_path: str) -> pd.DataFrame:
        """
        מחזירה את טבלת הזכויות מקובץ PDF כ־DataFrame.
        """
        # שלב 1: חיפוש עמוד תחילת הטבלה
        start_page = PdfTableExtractor._find_table_start_page(pdf_path)
        if start_page is None:
            raise ValueError("לא נמצאה טבלת זכויות בקובץ")

        # שלב 2: חילוץ שלוש שורות כותרת
        header_rows = PdfTableExtractor._extract_header_rows(pdf_path, start_page)
        column_names = PdfTableExtractor._generate_column_names(*header_rows)

        column_keywords = set(c.split(" / ")[-1] for c in column_names)

        # שלב 3: חילוץ שורות הנתונים
        data_rows = PdfTableExtractor._extract_data_rows(
            pdf_path, start_page, column_keywords, column_names
        )

        # שלב 4: בניית טבלה
        return pd.DataFrame(data_rows, columns=column_names)

    @staticmethod
    @contextlib.contextmanager
    def _suppress_stderr():
        with open(os.devnull, "w") as devnull:
            old_stderr = sys.stderr
            sys.stderr = devnull
            try:
                yield
            finally:
                sys.stderr = old_stderr

    @staticmethod
    def _find_table_start_page(pdf_path: str) -> int | None:
        search_phrase = "טבלת זכויות והוראות בניה"[::-1]  # חיפוש בטקסט הפוך
        with PdfTableExtractor._suppress_stderr():
            with pdfplumber.open(pdf_path) as pdf:
                for i, page in enumerate(pdf.pages):
                    text = page.extract_text()
                    if text and search_phrase in text:
                        return i
            return None

    @staticmethod
    def _extract_header_rows(
        pdf_path: str, start_page: int
    ) -> tuple[list[str], list[str], list[str]]:
        with PdfTableExtractor._suppress_stderr():
            with pdfplumber.open(pdf_path) as pdf:
                page = pdf.pages[start_page]
                tables = page.extract_tables()

                for table in tables:
                    if len(table) >= 3:
                        return table[0], table[1], table[2]

                raise ValueError("לא נמצאו שלוש שורות כותרת באף טבלה בעמוד")

    @staticmethod
    def _generate_column_names(row1: list, row2: list, row3: list) -> list[str]:
        """
        מחזירה שמות עמודות לפי שלוש שורות כותרת, עם היפוך טקסט מלא לעברית.
        """

        def clean(val):
            return str(val).replace("\n", " ").strip() if val else None

        def reverse(val):
            return val[::-1] if val else val

        def identify_split_columns_with_mapping(row: list) -> dict:
            split_columns = []
            main_columns = []
            split_to_main = {}
            temp_main = None
            temp_splits = []

            for idx, cell in enumerate(row):
                if cell is not None:
                    if temp_main is not None and temp_splits:
                        main_columns.append(temp_main)
                        for split_idx in temp_splits:
                            split_to_main[split_idx] = temp_main
                    temp_main = idx
                    temp_splits = []
                else:
                    split_columns.append(idx)
                    temp_splits.append(idx)

            if temp_main is not None and temp_splits:
                main_columns.append(temp_main)
                for split_idx in temp_splits:
                    split_to_main[split_idx] = temp_main

            return {
                "main_columns": main_columns,
                "split_columns": split_columns,
                "split_to_main": split_to_main,
            }

        mapping = identify_split_columns_with_mapping(row1)
        split_to_main = mapping["split_to_main"]

        column_names = []
        for idx in range(len(row1)):
            main_val = clean(row1[idx])
            mid_val = clean(row2[idx]) if idx < len(row2) else None
            sub_val = clean(row3[idx]) if idx < len(row3) else None

            if idx in split_to_main:
                main_idx = split_to_main[idx]
                main_val = clean(row1[main_idx])
                mid_val = clean(row2[idx]) or clean(row2[main_idx])
                sub_val = clean(row3[idx]) or clean(row3[main_idx])

            parts = [main_val, mid_val, sub_val]
            parts = [reverse(p) for p in parts if p]
            name = " / ".join(parts)
            column_names.append(name)

        return column_names

    @staticmethod
    def _extract_data_rows(
        pdf_path: str,
        start_page: int,
        column_keywords: set[str],
        original_header: list[str],
    ) -> list[list[str]]:
        import re

        def is_hebrew(text: str) -> bool:
            return bool(re.search(r"[\u0590-\u05FF]", text))

        def clean_cell(cell: str | None) -> str | None:
            if not cell:
                return None
            cell = str(cell).replace("\n", " ").strip()
            junk_words = ["מונה", "הנומ", "מונה הדפסה", "הצפסה הנומ"]
            for junk in junk_words:
                cell = cell.replace(junk, "").strip()
            return cell[::-1] if is_hebrew(cell) else cell if cell else None

        def is_end_of_table_row(row: list[str]) -> bool:
            joined = " ".join(filter(None, row)).strip()
            return "האמור בטבלה זו גובר" in joined

        rows = []

        with PdfTableExtractor._suppress_stderr():
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages[start_page:]:
                    tables = page.extract_tables()
                    for table in tables:
                        for row in table:
                            if row == original_header:
                                return rows  # עצירה על כותרת שחוזרת (טבלת המשך/נפרדת)

                            if is_end_of_table_row(row):
                                return rows

                            cleaned_row = [clean_cell(cell) for cell in row]
                            if len([c for c in cleaned_row if c]) < 3:
                                continue  # שורה חלשה — לא נכניס

                            rows.append(cleaned_row)

        return rows


pdf_path = r"C:\Users\dpere\Documents\python-projects\web-gis-scraper-desktop-electron\backend\101-0135004_תדפיס הוראות התכנית_חתום לאישור_1.pdf"
# pdf_path = r"C:\Users\dpere\Documents\python-projects\web-gis-scraper-desktop-electron\backend\101-0925198_תדפיס הוראות התכנית_חתום לאישור_1.pdf"
df = PdfTableExtractor.extract_table_from_pdf(pdf_path)

In [16]:
df

,")מ""ר( לשטח העיקרי מרפסות-בנוסף",מספר קומות / הקובעת מתחת לכניסה,מספר קומות / הקובעת מעל הכניסה,"מספר יח""ד",")מ""ר( שטחי בניה / מתחת לכניסה הקובעת / שרות",")מ""ר( שטחי בניה / מתחת לכניסה הקובעת / עיקרי",")מ""ר( שטחי בניה / מעל הכניסה הקובעת / שרות",")מ""ר( שטחי בניה / מתחת לכניסה הקובעת / עיקרי",")מ""ר( גודל מגרש / מוחלט גודל מגרש",בניין / מקום,תאי שטח,שימוש,יעוד
0,")מ""ר( לשטח העיקרי מרפסות-בנוסף",מספר קומות,None,"מספר יח""ד",")מ""ר( שטחי בניה",None,None,None,")מ""ר( גודל מגרש",בניין / מקום,תאי שטח,שימוש,יעוד
1,None,הקובעת מתחת לכניסה,הקובעת מעל הכניסה,None,מתחת לכניסה הקובעת,None,מעל הכניסה הקובעת,None,מוחלט גודל מגרש,None,None,None,None
2,None,None,None,None,שרות,עיקרי,שרות,עיקרי,None,None,None,None,None
3,300,2,8,(1) 100,2417,764,1946,5247,7220,"מלון הנביאים. בנין מס' 6,",1,מלונאי( מלונאות )אכסון,ותיירות מגורים מסחר
4,)2( תכ,None,2,None,None,None,None,392,7220,"נבון. בנין מס' 7, בית",1,מסחר ותעסוקה,ותיירות מגורים מסחר
5,None,None,4,None,None,None,33,969,7220,מבנה מס' 9,1,מסחר ותעסוקה,ותיירות מגורים מסחר
6,1776,None,29,(3) 148,None,None,4220,12650,7220,מבנה מס' 9.,1,מגורים,ותיירות מגורים מסחר
7,None,None,None,None,None,None,145,1325,7220,מבנה מס' 01,1,ציבור מבנים ומוסדות,ותיירות מגורים מסחר
8,None,None,5,None,None,None,750,3820,7220,מבנה מס' 01.,1,מסחר ותעסוקה,ותיירות מגורים מסחר
9,1764,None,31,(3) 147,None,None,4519,12635,7220,מבנה מס' 01,1,מגורים,ותיירות מגורים מסחר


In [8]:
import pdfplumber
import contextlib
import os
import sys
import pandas as pd
import re


class PdfTableExtractor:
    @staticmethod
    def extract_table_from_pdf(pdf_path: str) -> pd.DataFrame:
        """
        מחזירה את טבלת הזכויות מקובץ PDF כ־DataFrame.
        """
        # שלב 1: חיפוש עמוד תחילת הטבלה
        start_page = PdfTableExtractor._find_table_start_page(pdf_path)
        if start_page is None:
            raise ValueError("לא נמצאה טבלת זכויות בקובץ")

        # שלב 2: חילוץ שלוש שורות כותרת
        header_rows = PdfTableExtractor._extract_header_rows(pdf_path, start_page)
        column_names = PdfTableExtractor._generate_column_names(*header_rows)

        column_keywords = set(c.split(" / ")[-1] for c in column_names)

        # שלב 3: חילוץ שורות הנתונים
        data_rows = PdfTableExtractor._extract_data_rows(
            pdf_path, start_page, column_keywords
        )

        # שלב 4: בניית טבלה
        df = pd.DataFrame(data_rows, columns=column_names)

        # שלב 5: ניקוי הטבלה משורות ריקות או לא רלוונטיות
        df = PdfTableExtractor._clean_dataframe(df)

        return df

    @staticmethod
    @contextlib.contextmanager
    def _suppress_stderr():
        with open(os.devnull, "w") as devnull:
            old_stderr = sys.stderr
            sys.stderr = devnull
            try:
                yield
            finally:
                sys.stderr = old_stderr

    @staticmethod
    def _find_table_start_page(pdf_path: str) -> int | None:
        search_phrase = "טבלת זכויות והוראות בניה"[::-1]  # חיפוש בטקסט הפוך
        with PdfTableExtractor._suppress_stderr():
            with pdfplumber.open(pdf_path) as pdf:
                for i, page in enumerate(pdf.pages):
                    text = page.extract_text()
                    if text and search_phrase in text:
                        return i
            return None

    @staticmethod
    def _extract_header_rows(
        pdf_path: str, start_page: int
    ) -> tuple[list[str], list[str], list[str]]:
        with PdfTableExtractor._suppress_stderr():
            with pdfplumber.open(pdf_path) as pdf:
                page = pdf.pages[start_page]
                tables = page.extract_tables()

                for table in tables:
                    if len(table) >= 3:
                        return table[0], table[1], table[2]

                raise ValueError("לא נמצאו שלוש שורות כותרת באף טבלה בעמוד")

    @staticmethod
    def _generate_column_names(row1: list, row2: list, row3: list) -> list[str]:
        """
        מחזירה שמות עמודות לפי שלוש שורות כותרת, עם היפוך טקסט מלא לעברית.
        """

        def clean(val):
            return str(val).replace("\n", " ").strip() if val else None

        def reverse(val):
            return val[::-1] if val else val

        def identify_split_columns_with_mapping(row: list) -> dict:
            split_columns = []
            main_columns = []
            split_to_main = {}
            temp_main = None
            temp_splits = []

            for idx, cell in enumerate(row):
                if cell is not None:
                    if temp_main is not None and temp_splits:
                        main_columns.append(temp_main)
                        for split_idx in temp_splits:
                            split_to_main[split_idx] = temp_main
                    temp_main = idx
                    temp_splits = []
                else:
                    split_columns.append(idx)
                    temp_splits.append(idx)

            if temp_main is not None and temp_splits:
                main_columns.append(temp_main)
                for split_idx in temp_splits:
                    split_to_main[split_idx] = temp_main

            return {
                "main_columns": main_columns,
                "split_columns": split_columns,
                "split_to_main": split_to_main,
            }

        mapping = identify_split_columns_with_mapping(row1)
        split_to_main = mapping["split_to_main"]

        column_names = []
        for idx in range(len(row1)):
            main_val = clean(row1[idx])
            mid_val = clean(row2[idx]) if idx < len(row2) else None
            sub_val = clean(row3[idx]) if idx < len(row3) else None

            if idx in split_to_main:
                main_idx = split_to_main[idx]
                main_val = clean(row1[main_idx])
                mid_val = clean(row2[idx]) or clean(row2[main_idx])
                sub_val = clean(row3[idx]) or clean(row3[main_idx])

            parts = [main_val, mid_val, sub_val]
            parts = [reverse(p) for p in parts if p]
            name = " / ".join(parts)
            column_names.append(name)

        return column_names

    @staticmethod
    def _extract_data_rows(
        pdf_path: str, start_page: int, column_keywords: set[str]
    ) -> list[list[str]]:
        def is_hebrew(text: str) -> bool:
            return bool(re.search(r"[\u0590-\u05FF]", text))

        def clean_cell(cell: str | None) -> str | None:
            if not cell:
                return None
            cell = str(cell).replace("\n", " ").strip()
            junk_words = ["מונה", "הנומ", "מונה הדפסה", "הצפסה הנומ", "תכנון", "ןונכת"]
            for junk in junk_words:
                cell = cell.replace(junk, "").strip()
            return cell[::-1] if is_hebrew(cell) else cell if cell else None

        def is_end_of_table_row(row: list[str]) -> bool:
            joined = " ".join([str(c) for c in row if c]).strip()
            end_phrases = [
                "האמור בטבלה זו גובר",
                "רבוג וז הלבטב רומאה",
                "הערות לטבלת זכויות",
                "תויוכז תלבטל תורעה",
            ]
            return any(phrase in joined for phrase in end_phrases)

        def is_empty_row(row: list[str]) -> bool:
            """בדיקה אם השורה ריקה או מכילה רק תאים ריקים"""
            return all(cell is None or cell == "" for cell in row)

        def contains_data(row: list[str]) -> bool:
            """בדיקה אם השורה מכילה נתונים אמיתיים ולא רק כותרות או הערות"""
            # לפחות שלושה תאים עם נתונים
            data_cells = [cell for cell in row if cell]
            if len(data_cells) < 3:
                return False

            # בדיקה אם השורה מכילה מילות מפתח שלא אמורות להיות בנתונים
            joined = " ".join([str(c) for c in row if c]).strip().lower()
            junk_keywords = [
                "הערה",
                "הרעה",
                "המהווה חלק",
                "קלח הווהמה",
                "כפוף לתנאים",
                "םיאנתל ףופכ",
            ]
            if any(keyword in joined for keyword in junk_keywords):
                return False

            return True

        rows = []
        should_stop = False
        table_started = False

        with PdfTableExtractor._suppress_stderr():
            with pdfplumber.open(pdf_path) as pdf:
                page_index = start_page
                while page_index < len(pdf.pages) and not should_stop:
                    page = pdf.pages[page_index]
                    tables = page.extract_tables()

                    if not tables:
                        page_index += 1
                        continue

                    for table in tables:
                        if len(table) < 3:
                            continue

                        # בדיקה אם זו המשך של טבלת הזכויות
                        first_row = table[0]
                        is_continuation = any(
                            cell
                            and any(keyword in str(cell) for keyword in column_keywords)
                            for cell in first_row
                        )

                        # קביעת השורה להתחיל ממנה
                        if page_index == start_page and not table_started:
                            table_started = True
                            raw_rows = table[3:]  # דילוג על 3 שורות הכותרת בעמוד הראשון
                        elif is_continuation:
                            # אם זה המשך של טבלה - צריך לבדוק אם יש שורות כותרת
                            header_count = 0
                            for i, row in enumerate(table[:3]):
                                header_text = " ".join([str(c) for c in row if c])
                                if any(
                                    keyword in header_text
                                    for keyword in column_keywords
                                ):
                                    header_count = i + 1

                            raw_rows = table[header_count:]
                        else:
                            continue

                        for row in raw_rows:
                            if is_end_of_table_row(row):
                                should_stop = True
                                break

                            if is_empty_row(row):
                                continue

                            # ניקוי השורה
                            cleaned_row = [clean_cell(cell) for cell in row]

                            # הוספה רק אם זו שורת נתונים אמיתית
                            if contains_data(cleaned_row):
                                rows.append(cleaned_row)

                    page_index += 1

        return rows

    @staticmethod
    def _clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
        """ניקוי הטבלה משורות ריקות או לא רלוונטיות"""

        # מחיקת שורות עם יותר מדי תאים ריקים
        min_non_null = max(
            3, df.shape[1] // 3
        )  # לפחות שליש מהעמודות או 3 עמודות (הגבוה מביניהם)
        df = df.dropna(thresh=min_non_null)

        # מחיקת שורות כפולות
        df = df.drop_duplicates()

        # מחיקת שורות שמכילות רק מספרים (כמו מספרי עמודים)
        number_rows = df.apply(
            lambda row: all(
                str(x).isdigit() for x in row if x is not None and str(x).strip()
            ),
            axis=1,
        )
        df = df[~number_rows]

        # מחיקת שורות שמכילות טקסט מסוים שלא אמור להיות חלק מהטבלה
        def contains_junk_text(row):
            row_text = " ".join([str(x) for x in row if x is not None])
            junk_phrases = ["כפוף לתנאים", "המהווה חלק", "הערה", "הרעה"]
            return any(phrase in row_text for phrase in junk_phrases)

        df = df[~df.apply(contains_junk_text, axis=1)]

        # איפוס האינדקס
        df = df.reset_index(drop=True)

        return df


def extract_and_save_table(pdf_path: str, output_path: str = None) -> pd.DataFrame:
    """
    מחלץ את טבלת הזכויות מקובץ PDF ושומר כקובץ אקסל

    Args:
        pdf_path (str): נתיב לקובץ ה-PDF
        output_path (str, optional): נתיב לשמירת קובץ האקסל. אם לא מצוין,
                                    יישמר באותה תיקייה עם אותו שם אך סיומת xlsx

    Returns:
        pd.DataFrame: טבלת הזכויות כ-DataFrame
    """
    df = PdfTableExtractor.extract_table_from_pdf(pdf_path)

    return df


pdf_path = r"C:\Users\dpere\Documents\python-projects\web-gis-scraper-desktop-electron\backend\101-0135004_תדפיס הוראות התכנית_חתום לאישור_1.pdf"
df = extract_and_save_table(pdf_path)

# הצגת מספר שורות ראשונות
df

,")מ""ר( לשטח העיקרי מרפסות-בנוסף",מספר קומות / הקובעת מתחת לכניסה,מספר קומות / הקובעת מעל הכניסה,"מספר יח""ד",")מ""ר( שטחי בניה / מתחת לכניסה הקובעת / שרות",")מ""ר( שטחי בניה / מתחת לכניסה הקובעת / עיקרי",")מ""ר( שטחי בניה / מעל הכניסה הקובעת / שרות",")מ""ר( שטחי בניה / מתחת לכניסה הקובעת / עיקרי",")מ""ר( גודל מגרש / מוחלט גודל מגרש",בניין / מקום,תאי שטח,שימוש,יעוד
0,300,2,8,(1) 100,2417,764,1946,5247,7220,"מלון הנביאים. בנין מס' 6,",1,מלונאי( מלונאות )אכסון,ותיירות מגורים מסחר
1,)2( תכ,None,2,None,None,None,None,392,7220,"נבון. בנין מס' 7, בית",1,מסחר ותעסוקה,ותיירות מגורים מסחר
2,None,None,4,None,None,None,33,969,7220,מבנה מס' 9,1,מסחר ותעסוקה,ותיירות מגורים מסחר
3,1776,None,29,(3) 148,None,None,4220,12650,7220,מבנה מס' 9.,1,מגורים,ותיירות מגורים מסחר
4,None,None,None,None,None,None,145,1325,7220,מבנה מס' 01,1,ציבור מבנים ומוסדות,ותיירות מגורים מסחר
5,None,None,5,None,None,None,750,3820,7220,מבנה מס' 01.,1,מסחר ותעסוקה,ותיירות מגורים מסחר
6,1764,None,31,(3) 147,None,None,4519,12635,7220,מבנה מס' 01,1,מגורים,ותיירות מגורים מסחר
7,None,2,None,None,(4) 1327,(4) 8122,None,None,7220,הקובעת. מתחת לכניסה שטח משותף,1,מסחר ותעסוקה,ותיירות מגורים מסחר
8,None,6,None,None,(5) 42145,None,None,None,7220,None,1,חניון,ותיירות מגורים מסחר
9,273,None,8,(1) 91,None,None,1095,4126,2945,"מלון בוטיק. בנין מס' 2,",2,מלונאי( מלונאות )אכסון,ותיירות מגורים מסחר
